In [19]:
import requests
import pendulum
from pathlib import Path
import os
import sys

In [18]:
"""
Caso não exista a pasta 'downloads', ela é criada ao chamar este módulo
"""
#caminho_base = utils.dir_abs
diretorio_saida = Path(f'cfs_historico_full/')
diretorio_saida.mkdir(exist_ok=True, parents=True)


def auxiliar_download(url_base:str, data_requerida:pendulum.datetime, nome_arquivo:str) -> bool:
    """ Subfunção de download(). Não é chamada diretamente.
    Faz a requisição do dado de TSM no repositório da NOAA.
    Args:
        url_base (str): Parte imutável do link de endereço.
        data_requerida (pendulum.datetime): data do dado procurado.
        nome_arquivo (str): [description] Nome do arquivo a ser requisitado.
    Returns:
        bool: True = dado baixado, False= erro 404 | arquivo não disponível.
    """

    url=f'{url_base}/{nome_arquivo}'
    resp=requests.get(url)

    if resp.status_code == 200:
        resp=requests.get(url).content
        diretorio = Path(diretorio_saida, nome_arquivo)

        with open(diretorio, "wb") as arquivo_:
            arquivo_.write(resp)
            print(f"{nome_arquivo} [ok]")
            mensagem_de_sucesso = True

    elif resp.status_code == 404:
        print(f'Erro 404. {nome_arquivo} ainda não disponível')
        mensagem_de_sucesso = False

    return mensagem_de_sucesso


def download(data_string:str=None, hoje:bool=False) -> str:
    """Chama a função de download dos dados de temperatura oceânica do NOAA.
    Caso o dado final não esteja pronto, baixa o preliminar.
    Args:
        data_string (str, optional): [DD-MM-YYYY] data do dado procurado. Defaults to None.
        hoje (bool, optional): Habilita o dia atual na hora da execução. Sobrescreve o data_string. Defaults to False.
    Returns:
        str: Nome do arquivo baixado
    """

    data_requerida = pendulum.now('America/Sao_Paulo') if hoje else pendulum.from_format(data_string, 'DD-MM-YYYY')
    #url_base = 'https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr'
    url_base2='https://ftp.emc.ncep.noaa.gov/cmb/sst/oimonth_v2/GRIB'
    url_base='https://ftp.cpc.ncep.noaa.gov/precip/PORT/sst/oimonth_v2/GRIB/'

    nome_arquivo = f'oiv2mon.{data_requerida.format("YYYYMM")}.grb'	
    sucesso = auxiliar_download(url_base, data_requerida, nome_arquivo)
    if sucesso:
        print(f'{nome_arquivo} {sucesso}')
    else:
        print(f'>>> Tratativa de erro - link 2<<< {nome_arquivo}')
        sucesso = auxiliar_download(url_base2, data_requerida, nome_arquivo)
    '''
    if sucesso:
        tipo = "consistido"
        return nome_arquivo, tipo
    else :
        nome_arquivo =  f'oiv2mon.{data_requerida.format("YYYYMM")}198111.grb'	
        sucesso = auxiliar_download(url_base, data_requerida, nome_arquivo)
        
        if sucesso:
            tipo = "preliminar"
            return nome_arquivo, tipo
        else:
            sys.exit('A data solicitada ainda não possui dados')
    '''

In [ ]:
#import tsm
import pendulum

inicio = pendulum.datetime(1981,1,1)    #(1991, 1, 1)
fim = pendulum.datetime(2022, 8, 31)
periodo = pendulum.period(inicio, fim)

for dt in periodo.range('months'):
    data_string = dt.format('DD-MM-YYYY')

    download(data_string=data_string)

In [21]:
import xarray as xr

In [15]:
#ds = xr.open_dataset('download/oiv2mon.202012.grb')

In [ ]:
ds

In [22]:
diretorio_saida

PosixPath('cfs_historico_full')

In [23]:
dataset = xr.open_mfdataset(f'{diretorio_saida}/*.grb', concat_dim="valid_time", combine="nested",
                data_vars='minimal', coords='minimal', compat='override',
                backend_kwargs={
                    "filter_by_keys": {
                        "typeOfLevel": "surface",
                        "shortName": "t",
                    },
                 },
                parallel=True)

In [44]:
type(i.values)

numpy.datetime64

In [50]:
dataset.sel({"valid_time":[i.values]})

<xarray.Dataset>
Dimensions:     (latitude: 180, longitude: 360, valid_time: 1)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
  * longitude   (longitude) float64 -0.5 0.5 1.5 2.5 ... 355.5 356.5 357.5 358.5
  * valid_time  (valid_time) datetime64[ns] 2022-08-01
Data variables:
    t           (valid_time, latitude, longitude) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
Attributes:
    GRIB_edition:            1
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          4
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2022-09-23T16:46 GRIB to CDM+CF via cfgrib-0.9.1...

In [7]:
# para criar climatologia
#clima_tsm = dataset.t.groupby('valid_time.month').mean()

# esse aqui foi para criar histórico completo


In [ ]:
clima_tsm

<xarray.DataArray 't' (month: 12, latitude: 180, longitude: 360)>
dask.array<stack, shape=(12, 180, 360), dtype=float32, chunksize=(1, 180, 360), chunktype=numpy.ndarray>
Coordinates:
    time       datetime64[ns] ...
    step       timedelta64[ns] ...
    surface    float64 ...
  * latitude   (latitude) float64 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
  * longitude  (longitude) float64 -0.5 0.5 1.5 2.5 ... 355.5 356.5 357.5 358.5
  * month      (month) int64 1 2 3 4 5 6 7 8 9 10 11 12

In [36]:
clima_tsm.to_netcdf('oisst_climatologia_1982-2009.nc')

In [69]:
import recorta_dado
import pandas as pd
from regioes_indices import regioes

In [68]:
dicionario = {
    'data'   : [],
    "TSM Niño 1+2" : [],
    "TSM Niño 3" : [],
    "TSM Niño 3.4" : [],
    "TSM Niño 4" : [],
    "TSM CBM" : [],
    "TSM TNA" : [],
    "TSM TSA" : [],
}

In [14]:
for mes in range(1,12+1):
    tsm = clima_tsm.sel(month=mes)
    
    for indice in regioes:

        tsm_nino = recorta_dado.main(tsm,
            contorno_tipo='lat_lon',
            tipo='grib',
            latitude=[ indice.latitude[0], indice.latitude[1] ], 
            longitude=[ indice.longitude[0], indice.longitude[1] ]
            )

        valor = tsm_nino.mean(dim=["latitude", "longitude"]) 
        valor = valor - 273.15
        valor = round(float(valor.values), 2)

        dicionario[indice.nome].append(valor)

        print(f'O valor da região {indice.nome} no mês {mes} é : {valor}')

O valor da região TSM Niño 1+2 no mês 1 é : 24.32
O valor da região TSM Niño 3 no mês 1 é : 27.39
O valor da região TSM Niño 3.4 no mês 1 é : 26.48
O valor da região TSM Niño 4 no mês 1 é : 27.46
O valor da região TSM CBM no mês 1 é : 19.84
O valor da região TSM TNA no mês 1 é : 24.68
O valor da região TSM TSA no mês 1 é : 25.58
O valor da região TSM Niño 1+2 no mês 2 é : 25.87
O valor da região TSM Niño 3 no mês 2 é : 27.23
O valor da região TSM Niño 3.4 no mês 2 é : 26.67
O valor da região TSM Niño 4 no mês 2 é : 27.76
O valor da região TSM CBM no mês 2 é : 20.42
O valor da região TSM TNA no mês 2 é : 24.13
O valor da região TSM TSA no mês 2 é : 26.56
O valor da região TSM Niño 1+2 no mês 3 é : 26.2
O valor da região TSM Niño 3 no mês 3 é : 27.16
O valor da região TSM Niño 3.4 no mês 3 é : 27.18
O valor da região TSM Niño 4 no mês 3 é : 28.21
O valor da região TSM CBM no mês 3 é : 20.05
O valor da região TSM TNA no mês 3 é : 24.11
O valor da região TSM TSA no mês 3 é : 27.1
O valor d

In [ ]:
dicionario = {
    'data'   : [],
    "TSM Niño 1+2" : [],
    "TSM Niño 3" : [],
    "TSM Niño 3.4" : [],
    "TSM Niño 4" : [],
    "TSM CBM" : [],
    "TSM TNA" : [],
    "TSM TSA" : [],
}

for i in dataset.valid_time:
    
    ds = dataset.sel({"valid_time":[i.values]})
    
    data_str = str(i.values)[:10]
    data = pendulum.from_format(data_str, "YYYY-MM-DD")
    dicionario['data'].append(data.format('DD-MM-YYYY'))
    
    for indice in regioes:

        tsm_nino = recorta_dado.main(ds.t,
            contorno_tipo='lat_lon',
            tipo='grib',
            latitude=[ indice.latitude[0], indice.latitude[1] ], 
            longitude=[ indice.longitude[0], indice.longitude[1] ]
            )

        valor = tsm_nino.mean(dim=["latitude", "longitude"]) 
        valor = valor - 273.15
        valor = round(float(valor.values), 2)

        dicionario[indice.nome].append(valor)

        print(f'O valor da região {indice.nome} em {data.format("DD-MM-YYYY")} é : {valor}')

In [ ]:
dicionario

In [79]:
dt = pd.DataFrame.from_dict(dicionario)

In [83]:
dt

,data,TSM Niño 1+2,TSM Niño 3,TSM Niño 3.4,TSM Niño 4,TSM CBM,TSM TNA,TSM TSA
0,01-11-1981,21.10,26.80,25.99,27.08,15.43,26.78,24.00
1,01-12-1981,22.45,27.49,26.20,27.24,17.36,25.96,24.66
2,01-01-1982,24.01,27.09,26.68,27.37,19.07,24.81,25.39
3,01-02-1982,25.13,26.95,26.67,27.63,19.72,23.98,26.22
4,01-03-1982,24.71,26.77,27.18,27.98,20.39,23.97,26.75
...,...,...,...,...,...,...,...,...
485,01-04-2022,23.78,27.48,26.73,28.51,17.67,24.41,27.15
486,01-05-2022,22.24,27.67,26.71,28.18,15.80,25.17,26.23
487,01-06-2022,21.26,27.58,26.89,27.66,13.33,25.91,25.15
488,01-07-2022,20.45,27.50,26.53,26.99,11.91,26.24,23.68


In [82]:
dataset.to_netcdf('oiv2_historico_1981-2022.nc')

In [81]:
dt.set_index('data').to_csv('oiv2_historico_1981-2022.csv', sep=';')